# Replication Project: A quantitative theory of the gender gap in wages (Erosa, Fuster & Restuccia 2016)
Solving the dynamic model of the paper is a high dimensional problem that requires a lot of computational power. However, lot of computations can be made in parallel instead of sequentially. Consequently, we can shorten the running time significantly by relying on parallel computing packages. As recommended by Fernández-Villaverde & Zarruk (2018) we use the packages "Distributed" and "SharedArrays".
In a first step we load the packages and define the number of workers used in parallel computing. Note that we load the "Interpolations" package accompanied by the *@everywhere* command to make it accessible to all processing units used in the parallel computation.

For the rest of the code *@everywhere* is used to define objects that are unalterable and all workers of parallel computation need to access. Alterable variables and matrices are created via the *SharedArray* command.

In [1]:
using Plots, Distributed, SharedArrays, Distributions, QuantEcon, Interpolations#, Interpolations
addprocs(4) #Number of workers used in parallel computing
@everywhere using Interpolations

To increase speed additionally, we exploit properties of the Julia language. We do so by running computations within functions and using unalterable variables as parameter inputs for the model. For the latter we define a new type structure that summarizes all input parameters. We also include a function that helps us to create such a parameter structure with our desired values. The default input values are for the case of non-college females.

In [2]:
@everywhere struct Parameters
    α_1::Float64
    α_2::Float64
    γ_h::Float64
    intercept::Float64
    age1::Float64
    age2::Float64
    age3::Float64
    child::Float64
    interact::Float64
    #ρ::Float64
    #σ::Float64
    gp_ν_s::Int64
    grid_ν_s::Vector{Float64}
    gp_ν_c::Int64
    grid_ν_c::Vector{Float64}
    markov::Matrix{Float64}
    aux_ages::Array{Int64}
    aux_ν::Array{Float64}
    #int_ν_j = interpolate(knots, aux_ν, Gridded(Linear()))
    #min_h::Float64
    #max_h::Float64
    gp_h::Int64
    grid_h::Vector{Float64}
    gp_n::Int64
    grid_n::Vector{Float64}
    starting_age::Int64
    J::Int64
    β::Float64
    θ::Array{Float64,2}
    Δ::Float64
    μ_h::Float64
    σ_h::Float64
    μ_v_c::Float64
    hours_factor::Float64
    ρ::Float64
    σ::Float64

    function Parameters(α_1 = 0.351;
                        α_2 = 0.379,
                        γ_h = 1.0,
                        intercept = 0.196,
                        age1 = 3.32,
                        age2 = -0.086,
                        age3 = 0.0007,
                        child = -6.81,
                        interact = 0.133,
                        markov,
                        grid_ν_s,
                        gp_ν_s = 7,
                        μ_ν_c = 0.7,
                        gp_ν_c = 7,
                        aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65],
                        aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                        min_h = 0.0001,
                        max_h = 200,
                        gp_h = 101,
                        gp_n = 5,
                        starting_age = 17,
                        β = 0.99,
                        Δ = 3.0,
                        μ_h = 0.728,
                        σ_h = 0.233,
                        μ_v_c = 0.7,
                        ρ = 0.76,
                        σ = 0.79)
        grid_h = collect(range(min_h, stop = max_h, length=gp_h))
        grid_n = collect(range(0, stop = gp_n, length = gp_n))
        J = 4*(65-starting_age)
        percentile_ν_c = 0.9 # set percentile
        max_ν_c = -log(1-percentile_ν_c)/μ_ν_c
        grid_ν_c = collect(range(0, stop = max_ν_c, length=gp_ν_c))
        if starting_age==17 ##Non-college
            aux1=[fill(0.0269,5*4); fill(0.0265,5*4); fill(0.0265,5*4); fill(0.0090,8*4); fill(0.0,25*4)]
            aux2=[1.0 1.44 0.76 fill(0.76,gp_n-3)']
            θ = aux1*aux2
            hours_factor = 44.2/40.5 #from table 1
        elseif starting_age==20 ## College
            aux1=[fill(0.0082,5*4); fill(0.0210,5*4); fill(0.0259,5*4); fill(0.0086,5*4); fill(0.0,25*4)]
            aux2=[1.0 2.66 0.76 fill(1.27,gp_n-3)']
            θ = aux1*aux2
            hours_factor = 46.2/42.7 #from table 1
        end
        new(α_1, α_2, γ_h, intercept, age1, age2, age3, child, interact, gp_ν_s, grid_ν_s, gp_ν_c, grid_ν_c, markov, aux_ages, aux_ν, gp_h, grid_h, gp_n, grid_n, starting_age, J, β, θ, Δ, μ_h, σ_h, μ_v_c, hours_factor, ρ, σ)
    end
end

In the next field we define a few functions that are needed within the functions that will solve for the optimal policies later on. The former include primarily three value functions: $W^j(h,n,\nu)$, $H^j(h,n,\nu)$, and $V^j(h,n,\nu)$, which denote the value of working, staying at home, and the value of optimally deciding between these two options, respectively. Their functional form is defined on pages 174 and 175 in the paper. These values rely, among others, on the number of working hours $l(j,n)$, the value of optimal human capital accumulation $HCA(\cdot)$, and the cost of exerting effort $c(j,h)$. Consequently, these three functions are also defined.

Optimal human capital accumulation has a closed form solution. The function returns $HCA(\cdot)$ the value of optimal effort and optimal effort itself obtained from this closed form solution. The derivation is as follows:

\begin{align}
    \max_e c(j,h)log(1-e)+e\hat{V}^j(h(1+\Delta),n,\nu)&+(1-e)\hat{V}^j(h,n,\nu) \\
    -\frac{c(j,h)}{(1-e)} +\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu) &= 0 \\
    (1-e)(\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu)) &= c(j,h) \\
    1-e &= \frac{c(j,h)}{\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu)} \\
    e &= 1-\frac{c(j,h)}{\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu)}
\end{align}

In [3]:
@everywhere function c(j,h,P::Parameters) #cost function for exerting effort
    return (P.α_1+j^(P.α_2))*h^(P.γ_h)
end
@everywhere function HCA(c,V_hat,V_hat_delta) #Optimal human capital accumulation (HCA)
    e = max(1-c/(V_hat_delta-V_hat),0)        #closed form solution of optimal effort
    if e>1
        error("outside of definition")
    end
    return (c*log(1-e)+e*V_hat_delta+(1-e)*V_hat, e)
end

@everywhere function l(j,n,P::Parameters) #working hours depending on age "j" and number of children "n"
    if j < 40
        P.intercept+P.age1*j+P.age2*j^2+P.age3*j^3+P.child*n+P.interact*n*j
        else#working hours reduction due to children becomes obsolete after age 40
        P.intercept+P.age1*j+P.age2*j^2+P.age3*j^3
    end
end

@everywhere function W(j,h,n,ν,VV_hat,VV_hat_delta,ll,P::Parameters) #Value of working
    cc = c(j,h,P)
    (opt_HCA,e) = HCA(cc,VV_hat,VV_hat_delta)
    uu = h*ν
    aux_val = h*ll + (1-ll)*uu + P.γ_h*log(1+n) + opt_HCA
    return (aux_val, e)
end

@everywhere function H(j,h,n,ν,γ_n,VV_hat) #Value of staying at home
    uu = h*ν
    return uu + γ_n*log(1+n)+VV_hat
end

@everywhere function V(j,h,n,P::Parameters) #Value of optimal labor decision
    max(H(j,h,n,P::Parameters),V(j,h,n,P::Parameters))
end

Subsequently, we define the function that solves the policy functions for men. In a first bloc all the needed matrices are preallocated. We have three value functions for working, staying at home, and the optimal decision between the two. Furthermore, we have two policy functions: optimal decision of going to work (or not) and optimal exerted effort. Additionally, we have some auxiliary variables that need preallocation to perform parallel computing.

The state of a man is fully defined by the three variables: the level of human capital, the period withing the life-cycle and the stochastic value of staying at home $\nu_s$. The three variables then determine the size of the three dimensions that the policy and value functions have. Note that the paper only denotes two state variables for men: $\nu$ and the stock of human capital. However, for men $\nu$ has two dimensions as it equals $\nu=\nu_j \nu_s$. The notation of the paper therefore implicitely also includes the same three dimensions.

We use backwards induction to solve for all the combinations of human capital level and $\nu_s$ gridpoints. We start with the last period $J$, where the value is determined myopically. For all other periods, we use the value of the next period to include the dynamic component. Due to the stochastic components, the value of entering the next period is an expected value. The stochasticity has two components: First, whether the effort of human capital accumulation is successful. This only applies in the case of working. Second, the stochastic value of staying at home $\nu_s$ that follows an AR(1) process. We approximate the AR(1) process with the Tauchen method. This means that we discretize the continuous AR(1) process to a markov matrix with according gridpoints.

In [4]:
function male_policy(P::Parameters)
    #Preallocation
    #Value functions
    W_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Working
    H_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Staying at home
    V_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Optimal labor decision
    #policy functions
    labor_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #labor decision
    effort_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) # effort decision
    #initialisation of remaining variables
    age = SharedArray{Float64}(1)
    ll = SharedArray{Float64}(1)
    itp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s)
    etp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s)
    knots = (P.aux_ages,)
    int_ν_j = interpolate(knots, P.aux_ν, Gridded(Linear()))
    
    #Solving
    #Last period
    age = P.starting_age+P.J/4
    ll = P.hours_factor*l(age,0,P) #scale working hours up by exogenous factor
    for (ind_h,h) in enumerate(P.grid_h)
        for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
            ν = int_ν_j[age]*ν_s
            (W_value[ind_h,P.J,ind_ν], effort_policy[ind_h, P.J,ind_ν]) = W(P.J,h,0,ν,10^(-8),10^(-6),ll,P)
            H_value[ind_h,P.J,ind_ν]                                  = H(P.J,h,0,ν,P.γ_h,10^(-8))
            V_value[ind_h,P.J,ind_ν]                                  = max.(W_value[ind_h,P.J,ind_ν],H_value[ind_h,P.J,ind_ν])
            labor_policy[ind_h,P.J,ind_ν]                             = 1*(W_value[ind_h,P.J,ind_ν]>H_value[ind_h,P.J,ind_ν])
        end
    end
    #Backwards induction
    for j in P.J-1:-1:1
        #allocation of auxiliary variables and interpolations that are constant across states
        age = P.starting_age+j/4
        ll = P.hours_factor*l(age,0,P) #scale working hours up by exogenous factor
        println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s),V_value[:,j+1,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        #solving for all combinations of human capital and ν_s gridpoints
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            #preparing all inputs needed for later calculations
            ind_ν = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]       #current human capital
            h_delta = h*(1+(P.Δ/100)) #future human capital if accumulation is successful
            ν_s = P.grid_ν_s[ind_ν]   #stochastic component of staying at home
            contval = 0       #initialisation of expected value of next period if HK acc. is NOT successful
            contval_delta = 0 #initialisation of expected value of next period if HK acc. IS successful
            for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                contval       = contval      + P.markov[ind_ν,ind_ν_2]*etp_h[h,ν_s_2]
                contval_delta = contval_delta+ P.markov[ind_ν,ind_ν_2]*etp_h[h_delta,ν_s_2]
            end
            ν = int_ν_j[age]*ν_s     #value of staying at home: deterministic component times stochastic component
            #calculating the values and optimal policies
            (W_value[ind_h,j,ind_ν], effort_policy[ind_h, j,ind_ν]) = W(j, h, 0, ν, contval, contval_delta, ll, P)
            H_value[ind_h,j,ind_ν] = H(j,h,0,ν,P.γ_h, contval)
            V_value[ind_h,j,ind_ν] = max.(W_value[ind_h,j,ind_ν],H_value[ind_h,j,ind_ν])
            labor_policy[ind_h,j,ind_ν] = 1*(W_value[ind_h,j,ind_ν]>H_value[ind_h,j,ind_ν])
        end
    end
    return (W_value, H_value, V_value, labor_policy, effort_policy)
end

male_policy (generic function with 1 method)

The female case is an extension of the male case. The model assumes that females differ from males in the regard of their possibility of having children. Consequently, a choice variable is added to the problem, namely the decision to have an additional child (-> child_policy). This means that we include a third policy function for this decision and a fourth value, that denotes the expected value of entering a period and behaving optimally with regard to all choice variables, including the fertility decision. We denote this as B_value.

The addition of fertility decisions opens up two more dimensions in the state of a woman compared to a man. The (stochastic) value of enjoying a newborn child at home $\nu_c$ and the number of children $n$. For the former keep in mind that $\nu$ is defined of being $\nu=\nu_j(\nu_s+\nu_c)$. However, $\nu_c$ can only be enjoyed if a woman has a newborn child. Consequently, we have $\nu=\nu_j(\nu_s+\nu_c)$ if a woman has a newborn and otherwise we have $\nu=\nu_j\nu_s$, as in the male case.

The state of women is then fully defined in five dimensions (the first three are identical to the male case):
* human capital $h$
* age/period $j$
* the stochastic value of staying at home $\nu_s$
* the stochastic value of staying at home with a newborn child $\nu_c$
* the number of children $n$

In [5]:
function female_policy(P::Parameters)
    #Value functions
    W_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Working
    H_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Staying at home
    V_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Optimal labor decision
    B_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Continuation value
    #Policy functions
    labor_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #labor decision
    effort_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #effort decision
    child_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #fertility decision
    #initialisation of remaining variables
    age = SharedArray{Float64}(1)
    ll = SharedArray{Float64}(1)
    itp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s, P.gp_ν_c)
    etp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s, P.gp_ν_c)
    knots = (P.aux_ages,)
    int_ν_j = interpolate(knots, P.aux_ν, Gridded(Linear()))

    #Last period
    age = P.starting_age+P.J/4
    for (ind_n,n) in enumerate(P.grid_n)
        ll = l(age,n,P)
        for (ind_h,h) in enumerate(P.grid_h)
            for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
                ν = int_ν_j[age]*ν_s
                (WW, ee)                                              = W(P.J,h,n,ν,10^(-8),10^(-6),ll,P)
                HH                                                    = H(P.J,h,n,ν,P.γ_h,10^(-8))
                W_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*WW
                effort_policy[ind_h, P.J,ind_ν,:, ind_n]              = ones(1,P.gp_ν_c)*ee
                H_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*HH
                V_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*max.(WW,HH)
                B_value[ind_h,P.J,ind_ν,:,ind_n]                      = ones(1,P.gp_ν_c)*max.(WW,HH)
                labor_policy[ind_h,P.J,ind_ν,:, ind_n]                = ones(1,P.gp_ν_c)*(WW>HH)
            end
        end
    end
    #Backwards induction until 40 years -> irrelevant child decision
    for j in P.J-1:-1:((40-P.starting_age)*4)
        age = P.starting_age+j/4
        println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s,P.grid_ν_c,P.grid_n),B_value[:,j+1,:,:,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            ind_ν_s = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]
            h_delta = h*(1+(P.Δ/100))
            ν_s = P.grid_ν_s[ind_ν_s]
            ν = int_ν_j[age]*(ν_s+0)
            for (ind_n,n) in enumerate(P.grid_n)
                ll = l(age,n,P)
                contval = 0
                contval_delta = 0
                for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                    contval       = contval      + P.markov[ind_ν_s,ind_ν_2]*etp_h[h,ν_s_2,0,n]
                    contval_delta = contval_delta+ P.markov[ind_ν_s,ind_ν_2]*etp_h[h_delta,ν_s_2,0,n]
                end
                (WW, ee)                                        = W(j, h, 0, ν, contval, contval_delta, ll, P)
                HH                                              = H(j,h,0,ν,P.γ_h, contval)
                W_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*WW
                effort_policy[ind_h, j,ind_ν_s,:, ind_n]          = ones(1,P.gp_ν_c)*ee
                H_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*HH
                V_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*max.(WW,HH)
                B_value[ind_h,j,ind_ν_s,:,ind_n]                  = ones(1,P.gp_ν_c)*max.(WW,HH)
                labor_policy[ind_h,j,ind_ν_s,:, ind_n]            = ones(1,P.gp_ν_c)*(WW>HH)
            end
        end
    end
    #Backwards induction before 40 years -> relevant child decision
    for j in ((40-P.starting_age)*4-1):-1:1
        age = P.starting_age+j/4
        println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s,P.grid_ν_c,P.grid_n),B_value[:,j+1,:,:,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            ind_ν_s = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]
            h_delta = h*(1+(P.Δ/100))
            ν_s = P.grid_ν_s[ind_ν_s]
            for (ind_n,n) in enumerate(P.grid_n)
                ll = l(age,n,P)
                for (ind_ν_c, ν_c) in enumerate(P.grid_ν_c)
                    contval = 0
                    contval_delta = 0
                    for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                        contval       = contval      + P.markov[ind_ν_s,ind_ν_2]*etp_h[h,ν_s_2,ν_c,n]
                        contval_delta = contval_delta+ P.markov[ind_ν_s,ind_ν_2]*etp_h[h_delta,ν_s_2,ν_c,n]
                    end
                    ν = int_ν_j[age]*(ν_s+ν_c)
                    (W_value[ind_h,j,ind_ν_s,ind_ν_c,ind_n], effort_policy[ind_h, j,ind_ν_s,ind_ν_c, ind_n]) = W(j, h, 0, ν, contval, contval_delta, ll, P)
                    H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = H(j,h,0,ν,P.γ_h, contval)
                    V_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = max.(W_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n],H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n])
                    labor_policy[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = 1*(W_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n]>H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n])
                end
            end
        end
        @sync @distributed for ind in 1:((P.gp_n-1)*P.gp_ν_c)
            ind_ν_c = convert(Int, ceil(ind/(P.gp_n-1)))
            ind_n = convert(Int, floor(mod(ind-0.05, (P.gp_n-1)))+1)
            child_policy[:,j,:,ind_ν_c, ind_n] = 1*(V_value[:,j,:,ind_ν_c, ind_n+1] .> V_value[:,j,:,ind_ν_c, ind_n])
            B_value[:,j,:,ind_ν_c,ind_n] = P.θ[j,ind_n]*max.(V_value[:,j,:,ind_ν_c,ind_n+1], V_value[:,j,:,1,ind_n])+(1-P.θ[j,ind_n])*V_value[:,j,:,1, ind_n]
        end
    end
    return (W_value, H_value, V_value, B_value, labor_policy, effort_policy, child_policy)
end

female_policy (generic function with 1 method)

Now we allocate the the parameter collections for the college and non-college groups. We compute the discretized $\nu_s$ grid together with its markov matrix via the tauchen method, before feeding this into the function that creates the parameter collection. We do this so that we don't need to load the QuantEcon package into every worker used in parallel computing.

P_nonc denotes the parameter collection for non-college males and females. P_c denotes the one for college individuals.

In [6]:
ρ = 0.76
σ = 0.79
gp_ν_s = 7
m_non_c = QuantEcon.tauchen(gp_ν_s,ρ,0.79,0,1)
mmpp_non_c = m_non_c.p
@everywhere mp_non_c = $mmpp_non_c
expMM_non_c = exp.(collect(m_non_c.state_values))
@everywhere expM_non_c = $expMM_non_c

P_nonc = Parameters(0.351, α_2 = 0.379, γ_h = 1.0, intercept = 0.196, age1 = 3.32, age2 = -0.086, age3 = 0.0007, child = -6.81, interact = 0.133, markov = mp_non_c, grid_ν_s = expM_non_c, gp_ν_s = 7,
                aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65], aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                    min_h = 0.0001, max_h = 200, gp_h = 101, gp_n = 5, starting_age = 17, β = 0.99, Δ = 3.0, μ_h = 0.728, σ_h = 0.233, μ_v_c = 0.7,σ = 0.79)

Parameters(0.351, 0.379, 1.0, 0.196, 3.32, -0.086, 0.0007, -6.81, 0.133, 7, [0.296553, 0.444701, 0.666859, 1.0, 1.49957, 2.2487, 3.37208], 7, [0.0, 0.548235, 1.09647, 1.6447, 2.19294, 2.74117, 3.28941], [0.455081 0.200358 … 0.0191578 0.00711168; 0.307614 0.196478 … 0.0410885 0.0196142; … ; 0.0196142 0.0410885 … 0.196478 0.307614; 0.00711168 0.0191578 … 0.200358 0.455081], [17, 20, 25, 30, 40, 50, 55, 60, 65], [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6], 101, [0.0001, 2.0001, 4.0001, 6.0001, 8.0001, 10.0001, 12.0001, 14.0001, 16.0001, 18.0001  …  182.0, 184.0, 186.0, 188.0, 190.0, 192.0, 194.0, 196.0, 198.0, 200.0], 5, [0.0, 1.25, 2.5, 3.75, 5.0], 17, 192, 0.99, [0.0269 0.038736 … 0.020444 0.020444; 0.0269 0.038736 … 0.020444 0.020444; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 3.0, 0.728, 0.233, 0.7, 1.0913580246913581, 0.76, 0.79)

In [7]:
m_c = QuantEcon.tauchen(gp_ν_s,ρ,1.345,0,1)
mmpp_c = m_c.p
@everywhere mp_c = $mmpp_c
expMM_c = exp.(collect(m_c.state_values))
@everywhere expM_c = $expMM_c

P_c = Parameters(-0.31, α_2 = 0.457, γ_h = 1.0, intercept = -147.8, age1 = 16.09, age2 = -0.44, age3 = 0.0039, child = -12.56, interact = 0.239, markov = mp_c, grid_ν_s = expM_c, gp_ν_s = 7,
                aux_ages = [20, 21, 23, 25, 30, 40, 50, 60, 65], aux_ν = [35.10, 5.0, 0.35, 0.38, 0.07, 0.05, 0.05, 0.20, 0.86],
                    min_h = 0.0001, max_h = 200, gp_h = 101, gp_n = 5, starting_age = 20, β = 0.99, Δ = 4.15, μ_h = 0.976, σ_h = 0.395, μ_v_c = 4.1, σ=1.345)

Parameters(-0.31, 0.457, 1.0, -147.8, 16.09, -0.44, 0.0039, -12.56, 0.239, 7, [0.126252, 0.251667, 0.501664, 1.0, 1.99337, 3.97351, 7.92067], 7, [0.0, 0.548235, 1.09647, 1.6447, 2.19294, 2.74117, 3.28941], [0.455081 0.200358 … 0.0191578 0.00711168; 0.307614 0.196478 … 0.0410885 0.0196142; … ; 0.0196142 0.0410885 … 0.196478 0.307614; 0.00711168 0.0191578 … 0.200358 0.455081], [20, 21, 23, 25, 30, 40, 50, 60, 65], [35.1, 5.0, 0.35, 0.38, 0.07, 0.05, 0.05, 0.2, 0.86], 101, [0.0001, 2.0001, 4.0001, 6.0001, 8.0001, 10.0001, 12.0001, 14.0001, 16.0001, 18.0001  …  182.0, 184.0, 186.0, 188.0, 190.0, 192.0, 194.0, 196.0, 198.0, 200.0], 5, [0.0, 1.25, 2.5, 3.75, 5.0], 20, 180, 0.99, [0.0082 0.021812 … 0.010414 0.010414; 0.0082 0.021812 … 0.010414 0.010414; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 4.15, 0.976, 0.395, 4.1, 1.0819672131147542, 0.76, 1.345)

Now we solve the policy functions of males and females of both the college and the non-college group.

In [8]:
@time (W_value_m_c, H_value_m_c, V_value_m_c, labor_policy_m_c, effort_policy_m_c) = male_policy(P_c);
@time (W_value_m_nonc, H_value_m_nonc, V_value_m_nonc, labor_policy_m_nonc, effort_policy_m_nonc) = male_policy(P_nonc);

┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = male_policy(::Parameters) at In[4]:22
└ @ Main ./In[4]:22


64.75


┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##5#6")){Parameters,SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Int64})(::UnitRange{Int64}, ::Int64, ::Int64) at In[4]:45
└ @ Main ./In[4]:45
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##5#6")){Parameters,SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},Interpolations.GriddedInt

64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75
44.5
44.25
44.0
43.75
43.5
43.25
43.0
42.75
42.5
42.25
42.0
41.75
41.5
41.25
41.0
40.75
40.5
40.25
40.0
39.75
39.5
39.25
39.0
38.75
38.5
38.25
38.0
37.75
37.5
37.25
37.0
36.75
36.5
36.25
36.0
35.75
35.5
35.25
35.0
34.75
34.5
34.25
34.0
33.75
33.5
33.25
33.0
32.75
32.5
32.25
32.0
31.75
31.5
31.25
31.0
30.75
30.5
30.25
30.0
29.75
29.5
29.25
29.0
28.75
28.5
28.25
28.0
27.75
27.5
27.25
27.0
26.75
26.5
26.25
26.0
25.75
25.5
25.25
25.0
24.75
24.5
24.25
24.0
23.75
23.5
23.25
23.0
22.75
22.5
22.25
22.0
21.75
21.5
21.25
21.0
20.75
20.5
20.25
 77.033735 seconds (9

([0.358932 0.358673 … 0.00429091 0.00262263; 7178.99 7173.8 … 85.8093 52.4352; … ; 7.10684e5 7.10171e5 … 8494.7 5190.82; 7.17863e5 7.17345e5 … 8580.51 5243.26]

[0.355104 0.355102 … 0.00283603 0.00148523; 7102.42 7102.39 … 56.7132 29.6861; … ; 7.03105e5 7.03102e5 … 5614.32 2938.78; 7.10207e5 710204.0 … 5671.03 2968.46]

[0.349286 0.34968 … 0.000878634 -0.000220372; 6986.05 6993.95 … 17.5664 -4.42766; … ; 6.91585e5 6.92366e5 … 1738.98 -438.317; 6.98571e5 6.9936e5 … 1756.55 -442.745]

[0.34043 0.341434 … -0.00180999 -0.00277804; 6808.93 6829.01 … -36.2062 -55.5835; … ; 6.74051e5 6.76039e5 … -3584.23 -5502.5; 6.8086e5 6.82867e5 … -3620.44 -5558.08]

[0.32696 0.328899 … -0.00562117 -0.00661343; 6539.52 6578.29 … -112.432 -132.295; … ; 6.4738e5 6.51219e5 … -11130.2 -13096.6; 6.53919e5 6.57797e5 … -11242.6 -13228.9]

[0.30651 0.309878 … -0.0111765 -0.0123649; 6130.5 6197.86 … -223.543 -247.329; … ; 6.06889e5 6.13558e5 … -22129.7 -24484.4; 6.13019e5 6.19756e5 … -22353.2 -24731.7]

[0.275545 0

In [9]:
@time (W_value_f_c, H_value_f_c, V_value_f_c, B_value_f_c, labor_policy_f_c, effort_policy_f_c, child_policy_f_c) = female_policy(P_c);

┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = female_policy(::Parameters) at In[5]:25
└ @ Main ./In[5]:25


64.75


┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__

64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75
44.5
44.25
44.0
43.75
43.5
43.25
43.0
42.75
42.5
42.25
42.0
41.75
41.5
41.25
41.0
40.75
40.5
40.25
40.0
39.75


┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##8#11")){Parameters,SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Int64})(::UnitRange{Int64}, ::Int64, ::Int64) at In[5]:87
└ @ Main ./In[5]:87
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##8#11")){Parameters,SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},Interpolations.GriddedI

39.5
39.25
39.0
38.75
38.5
38.25
38.0
37.75
37.5
37.25
37.0
36.75
36.5
36.25
36.0
35.75
35.5
35.25
35.0
34.75
34.5
34.25
34.0
33.75
33.5
33.25
33.0
32.75
32.5
32.25
32.0
31.75
31.5
31.25
31.0
30.75
30.5
30.25
30.0
29.75
29.5
29.25
29.0
28.75
28.5
28.25
28.0
27.75
27.5
27.25
27.0
26.75
26.5
26.25
26.0
25.75
25.5
25.25
25.0
24.75
24.5
24.25
24.0
23.75
23.5
23.25
23.0
22.75
22.5
22.25
22.0
21.75
21.5
21.25
21.0
20.75
20.5
20.25
1145.831353 seconds (4.56 M allocations: 296.027 MiB, 0.01% gc time)


([1.33131 1.32796 … 0.0174197 0.00982531; 9259.13 9267.53 … 348.392 196.496; … ; 9.16609e5 9.1744e5 … 34489.1 19452.1; 9.25867e5 9.26707e5 … 34837.5 19648.6]

[1.32246 1.32111 … 0.0150755 0.00864873; 9082.09 9130.49 … 301.507 172.963; … ; 8.99082e5 9.03874e5 … 29847.7 17122.5; 9.08164e5 9.13004e5 … 30149.2 17295.5]

[1.30412 1.3071 … 0.0114847 0.00630338; 8715.21 8850.29 … 229.691 126.054; … ; 8.62763e5 8.76136e5 … 22738.3 12478.7; 8.71478e5 8.84985e5 … 22968.0 12604.8]

[1.26644 1.27859 … 0.00572046 0.00162822; 7961.43 8279.96 … 114.403 32.5461; … ; 7.88143e5 8.19675e5 … 11325.3 3221.91; 7.96104e5 827955.0 … 11439.7 3254.45]

[1.18965 1.22084 … -0.00423562 -0.00769107; 6425.52 7124.92 … -84.7256 -153.849; … ; 6.36095e5 7.05332e5 … -8387.42 -15230.3; 642520.0 7.12457e5 … -8472.15 -15384.1]

[1.03429 1.10445 … -0.0226317 -0.0262679; 3318.23 4796.98 … -452.662 -525.403; … ; 3.28489e5 4.74877e5 … -44811.3 -52012.3; 3.31807e5 4.79674e5 … -45263.9 -52537.7]

[0.72177 0.870824 … -0.0581267 -

In [12]:
@time (W_value_f_nonc, H_value_f_nonc, V_value_f_nonc, B_value_f_nonc, labor_policy_f_nonc, effort_policy_f_nonc, child_policy_f_nonc) = female_policy(P_nonc)

64.75
64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75
44.5
44.25
44.0
43.75
43.5
43.25
43.0
42.75
42.5
42.25
42.0
41.75
41.5
41.25
41.0
40.75
40.5
40.25
40.0
39.75
39.5
39.25
39.0
38.75
38.5
38.25
38.0
37.75
37.5
37.25
37.0
36.75
36.5
36.25
36.0
35.75
35.5
35.25
35.0
34.75
34.5
34.25
34.0
33.75
33.5
33.25
33.0
32.75
32.5
32.25
32.0
31.75
31.5
31.25
31.0
30.75
30.5
30.25
30.0
29.75
29.5
29.25
29.0
28.75
28.5
28.25
28.0
27.75
27.5
27.25
27.0
26.75


CompositeException: On worker 2:
outside of definition
error at ./error.jl:33
HCA at ./In[3]:7 [inlined]
W at ./In[3]:22
macro expansion at ./In[5]:91 [inlined]
#8 at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Distributed/src/macros.jl:291
#170 at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Distributed/src/macros.jl:43
#109 at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Distributed/src/process_messages.jl:265
run_work_thunk at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Distributed/src/process_messages.jl:56
run_work_thunk at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Distributed/src/process_messages.jl:65
#102 at ./task.jl:259
#remotecall_fetch#149(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Function, ::Distributed.Worker, ::Distributed.RRID, ::Vararg{Any,N} where N) at /Users/osx/buildbot/slav

In [28]:
function female_simulation(labor_policy, effort_policy, child_policy,P::Parameters)
    N = 1000 #number of simulations
    h_sim = SharedArray{Float64}(P.J+1,N)
    h_sim[1,:] = rand(LogNormal(P.μ_h, P.σ_h),N)
    ν_s_sim = SharedArray{Float64}(P.J+1,N)
    ν_s_sim[1,:] = ones(N)
    ν_c_sim = rand(Exponential(P.μ_v_c),(P.J,N))
    fertility_sim = SharedArray{Int64}(P.J,N)
    children_sim = SharedArray{Int64}(P.J+1,N)
    children_sim[1,:] = zeros(N)
    youngest_child_age = SharedArray{Int64}(P.J+1,N)
    #youngest_child_age[1,:] = fill(NaN,N)
    labor_sim = SharedArray{Int64}(P.J,N)
    unemployment_duration = SharedArray{Int64}(P.J+1,N) #in quarters
    unemployment_duration[1,:] = zeros(N)
    effort_sim = SharedArray{Float64}(P.J,N)
    hc_draw_sim = rand(Uniform(),(P.J,N)) #stochastic component to determine the success of HK accumulation
    innovation_sim = rand(Normal(P.μ_v_c),(P.J,N)) #innovation of AR1 process
    working_hours_sim = SharedArray{Float64}(P.J,N)
    child_sim = SharedArray{Int64}(P.J+1,N)
    itp_l = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s,P.grid_ν_c,P.grid_n),labor_policy[:,:,:,:,:],Gridded(Constant()))
    etp_l = extrapolate(itp_l, Flat())
    itp_c = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s,P.grid_ν_c,P.grid_n),child_policy[:,:,:,:,:],Gridded(Constant()))
    etp_c = extrapolate(itp_c, Flat())
    itp_e = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s,P.grid_ν_c,P.grid_n),effort_policy[:,:,:,:,:],Gridded(Linear()))
    etp_e = extrapolate(itp_e, Line())
    
    #@sync @distributed 
    for n in 1:N
        for j in 1:P.J
            age = P.starting_age+j/4
            ν_s_sim[j+1,n] = exp(P.ρ*log(ν_s_sim[j,n])+innovation_sim[j,n])
            prob = P.θ[j,min(child_sim[j,n]+1,P.gp_n)]
            if rand(Bernoulli(prob)) == 1 #woman is allowed to do fertility decision
                child = etp_c[h_sim[j,n],j,ν_s_sim[j+1,n],ν_c_sim[j+1,n],children_sim[j,n]]
                children_sim[j+1,n] = children_sim[j,n]+trunc(Int, child)
                if child == 1
                    fertility_sim[j,n] = 1
                    labor = etp_l[h_sim[j,n],j,ν_s_sim[j+1,n],ν_c_sim[j+1,n],children_sim[j+1,n]]
                    eff = etp_e[h_sim[j,n],j,ν_s_sim[j+1,n],ν_c_sim[j+1,n],children_sim[j+1,n]]
                else
                    labor = etp_l[h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n]]
                    eff = etp_e[h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n]]
                end
                #youngest_child_age[j+1,n] = youngest_child_age[j,n]+0.25
                labor_sim[j,n] = trunc(Int,labor)
                unemployment_duration[j+1,n] = (-1*trunc(Int,labor)+1)*(1+unemployment_duration[j,n])
                working_hours_sim[j,n] = labor*l(age, children_sim[j+1,n], P)
                effort_sim[j,n] = eff
                if eff >= hc_draw_sim[j,n]*labor
                    h_sim[j+1,n] = h_sim[j,n]*(1+P.Δ/100)
                else
                    h_sim[j+1,n] = h_sim[j,n]
                end
            else #woman is NOT allowed to do fertility decision-> ν_c = 0
                children_sim[j+1,n] = children_sim[j,n]
                labor = etp_l[h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n]]
                eff = etp_e[h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n]]
                labor_sim[j,n] = trunc(Int,labor)
                unemployment_duration[j+1,n] = (-1*trunc(Int,labor)+1)*(1+unemployment_duration[j,n])
                working_hours_sim[j,n] = labor*l(age, children_sim[j+1,n], P)
                effort_sim[j,n] = eff
                if eff >= hc_draw_sim[j,n]
                    h_sim[j+1,n] = h_sim[j,n]*(1+P.Δ/100)
                else
                    h_sim[j+1,n] = h_sim[j,n]
                end
            end
        end
    end
    return (h_sim[1:P.J,:], children_sim[2:P.J+1,:], labor_sim, working_hours_sim, fertility_sim, unemployment_duration[2:P.J+1,:])
end

female_simulation (generic function with 1 method)

In [29]:
(h_sim_f_c, child_sim_f_c, labor_sim_f_c, working_hours_sim_f_c, fertility_sim_f_c) = female_simulation(labor_policy_f_c, effort_policy_f_c, child_policy_f_c, P_c)

TypeError: TypeError: in Vararg, in count, expected Int64, got Float64

In [11]:
a = NaN
a = 1+a

NaN

In [23]:
fill(NaN,10)

10-element Array{Float64,1}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN